# Self-Driving Car Engineer Nanodegree


## Project 2: **Advanced Lane Lines** 
***
The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## Setup
---
### Import Packages

In [1]:
import numpy as np
import cv2
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


### Load Camera Calibrations
Camera calibration data generated in file [camera_calibration.ipynb](./camera_calibration.ipynb)

In [2]:
# load camera calibration data generated previously
cam_cal = pickle.load( open('./cam_cal_data.p', 'rb' ) )
mtx, dist = map(cam_cal.get, ('mtx', 'dist'))

### Load Perspective Transform Matrices
Perspective transform data generated in file [lane_pipeline.ipynb](./lane_pipeline.ipynb)

In [3]:
# load perspective tranform matrices generated previously
warp_data = pickle.load( open('./warp_data.p', 'rb' ) )
M, Minv = map(warp_data.get, ('M', 'Minv'))

### Support Functions
Sobel thresholds for generating binary mask

In [4]:
# applies Sobel x or y, then takes an absolute value and applies a threshold.
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Take the derivative in x or y given orient = 'x' or 'y'
    sobel = None
    if(orient == 'x'):
        sobel = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(img, cv2.CV_64F, 0, 1)
    # Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a mask of 1's where the scaled gradient magnitude 
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel > thresh_min) & (scaled_sobel < thresh_max)] = 1
            # is > thresh_min and < thresh_max
    # Return this mask as your binary_output image
    return sxbinary

# applies Sobel x and y, then computes the magnitude of the gradient and applies a threshold
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1)
    # Calculate the magnitude 
    abs_sobel = np.sqrt(np.square(sobelx) + np.square(sobely))
    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scale_factor = np.max(abs_sobel)/255 
    abs_sobel = (abs_sobel/scale_factor).astype(np.uint8) 
    # Create a binary mask where mag thresholds are met
    # Return this mask as your binary_output image
    binary_output = np.zeros_like(abs_sobel)
    binary_output[(abs_sobel >= mag_thresh[0]) & (abs_sobel <= mag_thresh[1])] = 1
    return binary_output

# applies Sobel x and y, then computes the direction of the gradient and applies a threshold.
def dir_thresh(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the x and y gradients
    abs_sobelx = np.sqrt(np.square(sobelx))
    abs_sobely = np.sqrt(np.square(sobely))
    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    grad_dir = np.arctan2(abs_sobely, abs_sobelx)
    # Create a binary mask where direction thresholds are metbinary_output =  np.zeros_like(absgraddir)
    binary_output = np.zeros_like(grad_dir)
    binary_output[(grad_dir >= thresh[0]) & (grad_dir <= thresh[1])] = 1
    # Return this mask as your binary_output image
    return binary_output

# applies binary mask directly related to the value of the channel
def binaryMask(img, thresh):
    binary = np.zeros_like(img)
    binary[(img > thresh[0]) & (img <= thresh[1])] = 1
    return binary

# combines a binary_mark on the r-channel (rgb) to various gradient thresholds applied to the l-channel (lab)
def apply_combined_threshold(image):
    
    # use r-channel to reduce confusion with shadows
    R = image[:,:,0]
    R_binary = binaryMask(R, (220, 255))
    
    # convert to hls and isolate s-channel
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    # use s-channel to emphasis yellow lines in light shaded roads
    l_channel = lab[:,:,0]
    
    grad_x = abs_sobel_thresh(l_channel, orient='x', thresh_min=20, thresh_max=120)
    grad_y = abs_sobel_thresh(l_channel, orient='y', thresh_min=20, thresh_max=120)
    grad_xy = np.zeros_like(grad_x)
    grad_xy[(grad_x == 1) & (grad_y == 1)] = 1
    
    mag_grad = mag_thresh(l_channel, sobel_kernel=5, mag_thresh=(30, 150))
    dir_grad = dir_thresh(l_channel, sobel_kernel=9, thresh=(0.7, 1.3))
    sobel_binary = np.zeros_like(grad_x)
    sobel_binary[(grad_xy == 1) | ((mag_grad == 1) & (dir_grad == 1))] = 1
    
    combined = np.zeros_like(R_binary)
    combined[(R_binary == 1) | (sobel_binary == 1)] = 1
    
    return combined

Function for finding the lanes in a binary, warped image and outputting the plotted points

In [5]:
def findLaneLine(binary_warped, nwindows=9, margin=100,minpix=50):

    # create historgram from sum across image pixels vertically
    histogram = np.sum(binary_warped, axis=0)

    # Find the peak of the left and right halves of the histogram for respective lanes
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # map all x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base# Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        # Find the four below boundaries of the window
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high) &
                (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)).nonzero()[0]
        good_right_inds = ((nonzerox >= win_xright_low) & (nonzerox < win_xright_high) &
                (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If found > minpix pixels, recenter next window
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty

Other support functions for drawing lanes on top of the image, as well as calculating curvature and car position data

In [6]:
def drawLane(img, left_fitx, right_fitx, Minv):
    # get y range for mapping x values
    ploty = np.linspace(0, img.shape[0] - 1, img.shape[0])
    
    # create blank image to draw lanes
    warped_lanes = np.zeros_like(img).astype(np.uint8)
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(warped_lanes, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    unwarped = cv2.warpPerspective(warped_lanes, Minv, (img.shape[1], img.shape[0])) 
    return cv2.addWeighted(img, 1, unwarped, 0.3, 0)

def measureCurvatureAndCarPosition(x_points, y_points, img, ym_per_pix=30/720, xm_per_pix=3.7/700):    
    # fit a second order polynomial in world space
    line_fit = np.polyfit(y_points*ym_per_pix, x_points*xm_per_pix, 2)

    # y space incompassess full top to bottom range image
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)

    # Calculation of curvature radius in world space
    line_curverad = ((1 + (2*line_fit[0]*y_eval*ym_per_pix + line_fit[1])**2)**1.5) / np.absolute(2*line_fit[0])
    
    #locate center of car based on center of image with respect to center of lane lines
    y_max = img.shape[0]*ym_per_pix
    x_max = img.shape[1]*xm_per_pix
    car_center = x_max/2
    
    line_mid = line_fit[0]*y_max**2 + line_fit[1]*y_max + line_fit[2]
    car_from_line = np.fabs(line_mid - car_center)
    
    return line_curverad, line_mid, car_from_line

### Lane Class
Class for maintaining lane stats and averaging for 'x' number of frames.

In [7]:
#hyperparameter for averaging frames
n_frame_hyper = 5

# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        # average x values of the fitted line over the last n iterations
        self.bestx = None
        #average y values of the fitted line over the last n iterations
        self.besty = None    
        #polynomial coefficients for the most recent fit
        self.current_fit = None
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None
        #line mid-point in world space
        self.line_mid = None
    
    # add new line data and update averages with latest lane found
    def addNewLineData(self, img, x_points, y_points, n_frames = n_frame_hyper):
        # update bestx with most recent points
        self.bestx = x_points
        self.besty = y_points

        # calculate current fit
        fit = np.polyfit(y_points, x_points, 2)
        ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
        # separate x-values for ploting on top of image
        fitx = fit[0]*ploty**2 + fit[1]*ploty + fit[2]
        # copy to current
        self.current_fit = np.copy(fitx)

        # append newest points
        self.recent_xfitted.append(fitx)

        # keep 'n' newest frames
        self.recent_xfitted = self.recent_xfitted[(0 - n_frames):]
        
        # calculate curvate of line's current fit
        self.radius_of_curvature, self.line_mid, self.line_base_pos = measureCurvatureAndCarPosition(x_points, y_points, img)
            
    def getAverageFit(self, weight):
        if len(self.recent_xfitted) == 0:
            return self.current_fit
        else:
            return (weight * self.current_fit) + ((1-weight) * sum(self.recent_xfitted)/len(self.recent_xfitted))

## Lane Finding Pipeline
---
Each frame/image undergoes the following process:

    undistort -> binary mask -> warp perspective -> find lanes -> draw lanes -> unwarp -> output

In [8]:
# takes image (or frame from video) and takes it through lane finding pipeline
# output: image with drawing of lane, as well as lane curvature information and car position
def process_image(img):
    global left_line
    global right_line
    
    # hyperparameter for calculating real word measurements
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # undistort image
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # apply threshold masks to create binary image
    binary = apply_combined_threshold(undist)
    # warp pespective to get top down view
    img_size = (undist.shape[1], undist.shape[0])
    binary_warped = cv2.warpPerspective(binary, M, img_size)
    
    # calculate lane line data points
    leftx, lefty, rightx, righty = findLaneLine(binary_warped, nwindows=8, margin=120,minpix=75)
    
    left_line.addNewLineData(binary_warped, leftx, lefty)
    right_line.addNewLineData(binary_warped, rightx, righty)
    
    # hyper parameter to weight most recent frame for averaging
    weight = 0.5
    
    # calculate averages for lane data
    left_fit_avg = left_line.getAverageFit(weight)
    right_fit_avg = right_line.getAverageFit(weight)
    
    # add lane to img
    lane_img = drawLane(undist, left_fit_avg, right_fit_avg, Minv)
    
    # calculate car position and curvature data
    lane_curvature = (left_line.radius_of_curvature + right_line.radius_of_curvature)/2
    lane_mid = (right_line.line_mid + left_line.line_mid)/2
    car_position = binary_warped.shape[1]/2 * xm_per_pix
    car_from_mid = np.fabs(lane_mid - car_position)
    car_side = 'left' if (left_line.line_base_pos < right_line.line_base_pos) else 'right'
    
    # add text to image
    cv2.putText(lane_img, 'Lane Curvature: {:04.2f}'.format(lane_curvature) + 'm',
               (50,75), cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255), 2)
    cv2.putText(lane_img, 'Car Position: {:04.3f}'.format(car_from_mid) + 'm ' + car_side + ' of center',
               (50,125), cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255), 2)
    
    return lane_img

## Process Test Video
---
Use './project_video' to take through lane finding pipelin

In [9]:
left_line = Line()
right_line = Line()

# load project video for testing
project_video = VideoFileClip('./project_video.mp4')

# put video frames through pipeline
out_video = project_video.fl_image(process_image) #NOTE: this function expects color images!!

# put together images and generate video
%time out_video.write_videofile('./out_video.mp4', audio=False)

[MoviePy] >>>> Building video ./out_video.mp4
[MoviePy] Writing video ./out_video.mp4


100%|█████████████████████████████████████▉| 1260/1261 [07:20<00:00,  2.86it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./out_video.mp4 

Wall time: 7min 21s
